##Get data

In [1]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "title contains 'flow_files_anon002_full.zip'"}).GetList()
for flow_file in listed:
  print('title {}, id {}'.format(flow_file['title'], flow_file['id']))

listed = drive.ListFile({'q': "title contains 'masks.zip'"}).GetList()
for mask_file in listed:
  print('title {}, id {}'.format(mask_file['title'], mask_file['id']))

listed = drive.ListFile({'q': "title contains 'anon002.zip'"}).GetList()
for video_file in listed:
  print('title {}, id {}'.format(video_file['title'], video_file['id']))

title flow_files_anon002_full.zip, id 1w1T69EEq_MvQUp5ju0_hbSLcsckeUDgN
title masks.zip, id 1JyhSp96W7cPy37UGugT8AXPeIjC6TMSI
title anon002.zip, id 1gc1LHkTElF1BNB5OfRjOczuvJHEAqWG8


In [2]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


file_id = flow_file['id']

fname = os.path.join('/content', 'flow_files.zip')
print('downloading to {}'.format(fname))
f_ = drive.CreateFile({'id': file_id})
f_.GetContentFile(fname)


file_id = mask_file['id']

fname = os.path.join('/content', 'masks.zip')
print('downloading to {}'.format(fname))
f_ = drive.CreateFile({'id': file_id})
f_.GetContentFile(fname)


file_id = video_file['id']

fname = os.path.join('/content', 'video.zip')
print('downloading to {}'.format(fname))
f_ = drive.CreateFile({'id': file_id})
f_.GetContentFile(fname)


downloading to /content/flow_files.zip
downloading to /content/masks.zip
downloading to /content/video.zip


In [3]:
!unzip flow_files.zip
!unzip masks.zip
!unzip video.zip

Archive:  flow_files.zip
   creating: flow_files_anon002_full/
  inflating: flow_files_anon002_full/000000.flo  
  inflating: flow_files_anon002_full/000001.flo  
  inflating: flow_files_anon002_full/000002.flo  
  inflating: flow_files_anon002_full/000003.flo  
  inflating: flow_files_anon002_full/000004.flo  
  inflating: flow_files_anon002_full/000005.flo  
  inflating: flow_files_anon002_full/000006.flo  
  inflating: flow_files_anon002_full/000007.flo  
  inflating: flow_files_anon002_full/000008.flo  
  inflating: flow_files_anon002_full/000009.flo  
  inflating: flow_files_anon002_full/000010.flo  
  inflating: flow_files_anon002_full/000011.flo  
  inflating: flow_files_anon002_full/000012.flo  
  inflating: flow_files_anon002_full/000013.flo  
  inflating: flow_files_anon002_full/000014.flo  
  inflating: flow_files_anon002_full/000015.flo  
  inflating: flow_files_anon002_full/000016.flo  
  inflating: flow_files_anon002_full/000017.flo  
  inflating: flow_files_anon002_full/

In [4]:
!pip uninstall opencv-python -y
# downgrade OpenCV a bit since some none-free features are not avilable
!pip install opencv-contrib-python==3.4.2.17 --force-reinstall

Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 30.6MB 139kB/s 
     |████████████████████████████████| 14.8MB 42.7MB/s 
ERROR: imgaug 0.2.9 requires opencv-python, which is not installed.
ERROR: dopamine-rl 1.0.5 requires opencv-python>=3.4.1.15, which is not installed.
ERROR: albumentations 0.1.12 requires opencv-python, which is not installed.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imageio
cv2.ocl.setUseOpenCL(False)
import sys
from PIL import Image
from pathlib import Path
from os import listdir

In [2]:
flowFileDirPath = '/content/flow_files_anon002_full'
fullImgDirPath = '/content/anon002'
transformation = "Affine"
mask_path = '/content/masks/anon002_mask.png'
padding_size = 1000

In [3]:
flowPaths =  [ flowFileDirPath + '/' + f  for f  in sorted(listdir(flowFileDirPath))]
fullImgPaths =  [ fullImgDirPath + '/' + f  for f  in sorted(listdir(fullImgDirPath))]

In [4]:
img_1 = cv2.imread(fullImgPaths[0])
img_1 = cv2.cvtColor(img_1,cv2.COLOR_BGR2RGB)

mask_im = Image.open(mask_path)
mask_im = mask_im.resize((img_1.shape[0], img_1.shape[1]), Image.ANTIALIAS)
mask_im = np.array(mask_im) * np.uint8(255)
mask_im = cv2.resize(mask_im, img_1.shape[1::-1])

In [5]:
def inputAndVisualizeStitchPair(srcImgPath, destImgPath, showImages=True): 
  '''
    Inputs: srcImgPath, destImgPath 
    srcImgPath : path to the image been transformed, usually the new frame seen ->str
    destImgPath : path to the image that acts as destination(src image is trasformed to it), usually the old frame->str
    Output: [srcImg, destImg] -> list
    srcImg: 256x256 representation of srcImage ->ndarray
    destImg: 256x256 representation of destImage ->ndarray
  '''
  srcImg = cv2.imread(srcImgPath)
  srcImg = cv2.cvtColor(srcImg,cv2.COLOR_BGR2RGB)
  srcImg_gray = cv2.cvtColor(srcImg, cv2.COLOR_RGB2GRAY)

  destImg = cv2.imread(destImgPath)
  destImg = cv2.cvtColor(destImg,cv2.COLOR_BGR2RGB)
  destImg_gray = cv2.cvtColor(destImg, cv2.COLOR_RGB2GRAY)

  if showImages:
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, constrained_layout=False, figsize=(10,4))
    ax1.imshow(destImg, cmap="gray")
    ax1.set_xlabel("dest image", fontsize=14)

    ax2.imshow(srcImg, cmap="gray")
    ax2.set_xlabel("Src image (Image to be transformed)", fontsize=14)
    
    plt.show()

  return [srcImg, destImg]

In [8]:
def inputAndFormatFlowfile(flowPath):
  '''
  Input: flowPath
  flowpath: path to the .flo file between two images ->str
  Output: flow
  flow: the formated optical_flow -> ndArray of heightxbreathx2 of the image sizes 
  '''
  path = Path(flowPath) 
  with path.open(mode='r') as flo:
      np_flow = np.fromfile(flo, np.float32)
      # print(np_flow.shape)

  with path.open(mode='r') as flo:
    tag = np.fromfile(flo, np.float32, count=1)[0]
    width = np.fromfile(flo, np.int32, count=1)[0]
    height = np.fromfile(flo, np.int32, count=1)[0]

    print('tag', tag, 'width', width, 'height', height)

    nbands = 2
    tmp = np.fromfile(flo, np.float32, count= nbands * width * height)
    flow = np.resize(tmp, (int(height), int(width), int(nbands)))
    return flow

In [9]:
def getCameraPixels(img):

  img = cv2.bitwise_and(img, img, mask=mask_im)
  # _,  imgMask = cv2.threshold(img,5,255,cv2.THRESH_BINARY)

  imgMaskGray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)


  nonZero = cv2.findNonZero(imgMaskGray)
  I,J = np.transpose(nonZero).squeeze()

  return [I, J]

In [10]:
def pointCorrespondenceFromOpticalFlow(flow, padding, nonZeroI, nonZeroJ):
  '''
  Input: flow
  flow: the formatted optical flow file -> ndarray length,breadth,2
  padding: padding to add to destImg
  OutPut: [ptsA and ptsB] -> list
  ptsA: point correspondences in the destImg (original) -> ndarray length*breadth,2
  ptsB: point correspondences in the srcImg(new, been transformed) -> ndarray length*breadth,2
  '''
  ptsA = np.zeros(flow.shape)
  ptsB = np.zeros(flow.shape)


  for i in range(ptsA.shape[0]):
    for j in range(ptsA.shape[1]):
      ptsA[i,j] = np.array([i,j], dtype=np.float)
      ptsB[i,j] = np.array([i,j])  +  (np.array(flow[j, i]))

  # print("before ptsA", ptsA.shape)
  # print("check ptsA", ptsA[217, 8] )
  # print("check ptsB", ptsB[217, 8] )
  
  ptsA = ptsA[nonZeroI,nonZeroJ]
  ptsB = ptsB[nonZeroI,nonZeroJ]

  # print("after ptsA", ptsA.shape)
  # print("after check ptsA", ptsA[0] )

  return [ptsA, ptsB]

In [11]:
def findTransformation(ptsA, ptsB, threshold=1):
  '''
    ptsA: point correspondences in the destImg (original) -> ndarray length*breadth,2
    ptsB: point correspondences in the srcImg(new, been transformed) -> ndarray length*breadth,2
    threshold: ransac threshold. 
    Output:
    H - homography
    status - mask of ransac accepted or rejected
  '''
  if transformation == "Homography":
    (H, status) = cv2.findHomography(ptsB, ptsA, cv2.RANSAC, threshold)
  elif transformation == "Affine":
    (H, status) = cv2.estimateAffine2D(ptsB, ptsA, method = cv2.RANSAC, ransacReprojThreshold = threshold)
  return [H, status]

In [12]:
def visualizeStitch(srcImg, destImg, H, padding, showImages=True):
  '''
  srcImg: 256x256 representation of srcImage ->ndarray
  destImg: 256x256 representation of destImage ->ndarray
  H - homography
  '''
  ht, wd, cc= destImg.shape

  ww = wd + (2*padding)
  hh = ht + (2*padding)

  xx = (ww - wd) // 2
  yy = (hh - ht) // 2

  if transformation == "Homography":
    result = cv2.warpPerspective(srcImg, H, (ww, hh))
  elif transformation == "Affine":
    result = cv2.warpAffine(srcImg, H, (ww, hh))

  alpha_s = mask_im/ 255.0
  alpha_l = 1.0 - alpha_s

  for c in range(0, 3):
      result[yy:yy+ht, xx:xx+wd, c] = (alpha_s * destImg[:, :, c] +
                                alpha_l * result[yy:yy+ht, xx:xx+wd, c])
  
  print("Visualize stitch")
  if showImages:
    plt.figure(figsize=(10,4))
    plt.imshow(result)

    plt.show()

In [ ]:
# b = np.arange(16).reshape((4,4))
# b_3d = b[:, :, None] * np.ones(3, dtype=int)[None, None, :]
# print(b)
# print(1-b_3d[:,:,2])

In [13]:
image_width = 256
def VisualizeOutliers(destImgPath, status, showImages):
  if showImages:
    ransac_fail = np.where(np.any(status==0, axis=1))[0]
    convert_to_pixel = lambda t: [t//image_width, t%image_width]
    outlier_pixels = np.array([convert_to_pixel(p) for p in ransac_fail])
    # print(outlier_pixels)
    destImg = Image.open(destImgPath)
    fig = plt.figure(figsize=(10,4))
    plt.xlabel("outliers",fontsize=14)
    if (outlier_pixels.size):
      plt.plot(outlier_pixels[:, 1],outlier_pixels[:, 0],'r.')
    plt.imshow(destImg)

In [14]:
def plotPixels(img, I, J):
  fig = plt.figure(figsize=(10,4))
  plt.plot(I,J,'r.')
  plt.imshow(img)
  plt.show()

#### Use functions to perform first stitch.

In [ ]:
# srcImgPath = '/content/video01/anon001_00852.png'
# destImgPath = '/content/video01/anon001_00851.png'
# flowPath = '/content/flow_files_video01_full/000000.flo'
# padding = 0
# threshold = 7
# showImages = True

# srcImg, destImg = inputAndVisualizeStitchPair(srcImgPath, destImgPath, showImages)
# flow = inputAndFormatFlowfile(flowPath)

# I, J = getCameraPixels(destImg)
# # plotPixels(destImg, I, J)

# ptsA, ptsB =  pointCorrespondenceFromOpticalFlow(flow, padding, I , J)
# H, status = findTransformation(ptsA, ptsB, threshold)

# print(H)

# visualizeStitch(srcImg, destImg, H, padding, showImages)

### mosaic registration

I used a crude way to make the mosaic here. I placed the first image into a very big blank canvas, then warped all the consecutive images towards it. 

There is a better method where you can choose whichever image you want as reference frame (such as the center image). This is shown in the get_lucas_kanade_mosaic.ipynb file and the global_registration_ongoing.ipynb file 

In [15]:
def pointCorrespondenceWithoutPadding(flow, nonZeroI, nonZeroJ):
  '''
  Input: flow
  flow: the formatted optical flow file -> ndarray length,breadth,2
  padding: padding to add to destImg
  OutPut: [ptsA and ptsB] -> list
  ptsA: point correspondences in the destImg (original) -> ndarray length*breadth,2
  ptsB: point correspondences in the srcImg(new, been transformed) -> ndarray length*breadth,2
  '''
  ptsA = np.zeros(flow.shape)
  ptsB = np.zeros(flow.shape)

  for i in range(ptsA.shape[0]):
    for j in range(ptsA.shape[1]):
      ptsA[i,j] = np.array([i,j], dtype=np.float)
      ptsB[i,j] = np.array([i,j])  +  (np.array(flow[j,i]))

  ptsA = ptsA[nonZeroI,nonZeroJ]
  ptsB = ptsB[nonZeroI,nonZeroJ]

  ptsA = np.reshape(ptsA, (-1, 2))
  ptsB = np.reshape(ptsB, (-1, 2))
  return [ptsA, ptsB]

In [17]:
def pointCorrespondenceWithPaddingForFirst(flow, nonZeroI, nonZeroJ, padding= padding_size):
  '''
  Input: flow
  flow: the formatted optical flow file -> ndarray length,breadth,2
  padding: padding to add to destImg
  OutPut: [ptsA and ptsB] -> list
  ptsA: point correspondences in the destImg (original) -> ndarray length*breadth,2
  ptsB: point correspondences in the srcImg(new, been transformed) -> ndarray length*breadth,2
  '''
  ptsA = np.zeros(flow.shape)
  ptsB = np.zeros(flow.shape)

  for i in range(ptsA.shape[0]):
    for j in range(ptsA.shape[1]):
      ptsA[i,j] = np.array([i,j], dtype=np.float) + np.array([padding,padding])
      ptsB[i,j] = np.array([i,j])  +  (np.array(flow[j,i]))


  ptsA = ptsA[nonZeroI,nonZeroJ]
  ptsB = ptsB[nonZeroI,nonZeroJ]

  ptsA = np.reshape(ptsA, (-1, 2))
  ptsB = np.reshape(ptsB, (-1, 2))
  return [ptsA, ptsB]

In [18]:
padding = padding_size
threshold = 7
if transformation == "Homography":
  H_array = np.zeros((len(fullImgPaths)-1, 3,3 ))
elif transformation == "Affine":    
  H_array = np.zeros((len(fullImgPaths)-1, 2,3 ))
showImages = False

In [19]:
for i in range(len(fullImgPaths) - 1): #
  destImgPath = fullImgPaths[i] #the previous image
  srcImgPath =  fullImgPaths[i + 1];
  flowPath = flowPaths[i]

  srcImg, destImg = inputAndVisualizeStitchPair(srcImgPath, destImgPath, showImages)
  flow = inputAndFormatFlowfile(flowPath)
  I, J = getCameraPixels(destImg)

  if i==0:
    ptsA, ptsB = pointCorrespondenceWithPaddingForFirst(flow, I, J, padding)
  else:
    ptsA, ptsB = pointCorrespondenceWithoutPadding(flow, I, J)
  H, status = findTransformation(ptsA, ptsB, threshold)
  if i==0:
    visualizeStitch(srcImg, destImg, H, padding, showImages)
  else:
    visualizeStitch(srcImg, destImg, H, 0, showImages)

  H_array[i] = H

tag 202021.25 width 512 height 512


IndexError: ignored

In [ ]:
H_array

array([[[ 9.98984573e-01,  1.75080357e-02,  9.93936478e+02],
        [-1.17374084e-02,  9.97045965e-01,  1.00138521e+03]],

       [[ 1.00349058e+00,  1.69731943e-02, -7.79437433e+00],
        [-3.21567142e-03,  1.00755193e+00, -2.57728448e+00]],

       [[ 1.00040670e+00,  2.41963286e-02, -1.29477290e+01],
        [-1.23762075e-02,  1.01161244e+00, -7.13711270e+00]],

       [[ 1.00850485e+00,  4.71653904e-04, -9.55962143e+00],
        [-2.61926231e-04,  1.00390680e+00, -1.27676030e+01]],

       [[ 1.00248666e+00,  7.79684871e-03, -7.62591505e+00],
        [ 1.25624238e-03,  1.00474119e+00, -1.35965299e+01]],

       [[ 1.01371220e+00,  4.06457071e-02, -1.97782651e+01],
        [-9.31065418e-04,  1.02186224e+00, -1.45509891e+01]],

       [[ 1.01314253e+00,  3.57943286e-02, -2.75666603e+01],
        [-7.61742330e-03,  1.01583733e+00, -7.05882107e+00]],

       [[ 9.98631357e-01,  9.59689202e-02, -3.67038088e+01],
        [-2.05501217e-02,  1.01923775e+00, -1.93083857e+00]],

       [

In [ ]:
H_global =  np.zeros((len(fullImgPaths)-1, 3,3 ))

In [ ]:
def getHGlobalRecursively(pos = 0, new_H = np.eye(3)):
  if pos == H_array.shape[0]:
    return 1
  if transformation == "Homography":
    new_H = np.matmul(new_H, H_array[pos] ) 
  elif transformation == "Affine":  
    new_H = np.matmul(new_H, np.vstack([H_array[pos], [0,0,1]]) )  
  
  H_global[pos] = new_H
  pos = pos+1;
  getHGlobalRecursively(pos,new_H)


getHGlobalRecursively()

In [ ]:
# import shutil

# shutil.rmtree('globalRegistrationTransparent')

In [ ]:
!mkdir globalRegistrationTransparent

In [ ]:
def globalImageRegistration(srcImg, destImg, index, imageName, padding=padding_size):
  ht, wd, cc= destImg.shape

  ww = wd + (2*padding)
  hh = ht + (2*padding)

  xx = (ww - wd) // 2
  yy = (hh - ht) // 2

  srcImg = cv2.bitwise_and(srcImg, srcImg, mask=mask_im)
  
  result = cv2.warpPerspective(srcImg, H_global[index], (ww, hh))

  return result

In [ ]:
def getTransparentImg(src, imageName):
  tmp = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)
  _,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
  r, g, b = cv2.split(src)
  rgba = [b,g,r, alpha]
  dst = cv2.merge(rgba,4)
  save_name = "/content/globalRegistrationTransparent/" + imageName
  cv2.imwrite(save_name, dst)

In [ ]:
def globalRegistrationFirstImage(img, imageName, padding=padding_size):
  ht, wd, cc= img.shape

  ww = wd + (2*padding)
  hh = ht + (2*padding)

  color = (0,0,0)
  result = np.full((hh,ww,cc), color, dtype=np.uint8)

  xx = (ww - wd) // 2
  yy = (hh - ht) // 2

  # copy img image into center of result image
  
  img =cv2.bitwise_and(img, img, mask=mask_im)
  
  result[yy:yy+ht, xx:xx+wd] = img

  return result

In [ ]:
# globalImageRegistration(srcImg, destImg, 7, "anon001_02320.png") 
# getTransparentImg("anon001_02320.png")

In [ ]:
for i in range(len(fullImgPaths) - 1):
  destImgPath = fullImgPaths[0] #the first image
  srcImgPath =  fullImgPaths[i + 1];
  srcImg, destImg = inputAndVisualizeStitchPair(srcImgPath, destImgPath, False)
  if (i == 0):
    # add reference image which is the first image
    imageName = Path(fullImgPaths[i]).name
    src = globalRegistrationFirstImage(destImg, imageName)
    getTransparentImg(src, imageName)

  imageName = Path(fullImgPaths[i + 1]).name
  src = globalImageRegistration(srcImg, destImg, i, imageName)
  getTransparentImg(src, imageName)

In [ ]:
!zip -r /content/globalRegistration.zip /content/globalRegistrationTransparent

updating: content/globalRegistrationTransparent/ (stored 0%)
  adding: content/globalRegistrationTransparent/anon010_02092.png (deflated 17%)
  adding: content/globalRegistrationTransparent/anon010_02089.png (deflated 17%)
  adding: content/globalRegistrationTransparent/anon010_02100.png (deflated 18%)
  adding: content/globalRegistrationTransparent/anon010_02114.png (deflated 17%)
  adding: content/globalRegistrationTransparent/anon010_02123.png (deflated 17%)
  adding: content/globalRegistrationTransparent/anon010_02137.png (deflated 18%)
  adding: content/globalRegistrationTransparent/anon010_02115.png (deflated 17%)
  adding: content/globalRegistrationTransparent/anon010_02083.png (deflated 18%)
  adding: content/globalRegistrationTransparent/anon010_02145.png (deflated 17%)
  adding: content/globalRegistrationTransparent/anon010_02087.png (deflated 18%)
  adding: content/globalRegistrationTransparent/anon010_02134.png (deflated 16%)
  adding: content/globalRegistrationTransparent/